# Portfolio's Building #

### Rebalancing Weights ###

In [1]:
# Import Libraries

# Data Management
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt

# Handle Files
import sys
import os

# Import Local Functions
sys.path.append(os.path.abspath("../source"))
from config import get_tickers
from data_downloader import get_market_data
from portfolios_toolkit import calculate_analytics
from portfolios_toolkit import markowitz_weights

In [2]:
tickers = get_tickers(mod="2.4")

tickers

In [3]:
# Import data
df_returns = pd.DataFrame()

for ticker in tickers:
    df = get_market_data(
        ticker=ticker, 
        start_date='2015-01-01', 
        end_date='2025-01-01', 
        returns=True
    )
    
    returns = df['returns'].rename(ticker)
    
    df_returns = pd.concat([df_returns, returns], axis=1)
    
    print(f'Data Ready for {ticker}')

In [4]:
df_returns

In [5]:
df_returns.mean()

In [6]:
# Time Series Graphs
plt.figure(figsize=(10, 6))
plt.plot(df_returns.cumsum(), label=df_returns.columns, alpha=1)

# Config
plt.title('Cumulative Returns Time Series')
plt.xlabel('Time Index')
plt.ylabel('Cumulative Returns')
plt.legend()

# Show
plt.grid(True)
plt.show()

In [7]:
# Let us Calculate the Weights
def rolling_weights(
    returns, 
    desired_returns, 
    window=252, 
    rebalance_freq=126
):

    # Lists to Store Things
    weights_list = []
    dates = []

    for i in range(window, len(returns), rebalance_freq):
        past_returns = returns.iloc[i - window:i]  # Rolling Window
        past_excepted_returns = past_returns.mean()
        past_cov_matrix = past_returns.cov()

        # Calculate Weights
        w = markowitz_weights(past_excepted_returns, past_cov_matrix, desired_returns)

        # Save weights and dates
        weights_list.append(w)
        dates.append(returns.index[i])

    # Create the DataFrame
    weights_df = pd.DataFrame(weights_list, index=dates, columns=returns.columns)

    # Expand the DataFrame
    weights_df = weights_df.reindex(returns.index, method='ffill')

    return weights_df.dropna()

In [8]:
# Create the DataFrames of Returns
df_weights = rolling_weights(df_returns, 0.002)

df_weights

In [9]:
# Time Series Graphs
plt.figure(figsize=(10, 6))
plt.plot(df_weights, label=df_weights.columns, alpha=1)
# Config
plt.title('Weights Time Series')
plt.xlabel('Time Index')
plt.ylabel('Weights')
plt.legend()

# Show
plt.grid(True)
plt.show()

In [10]:
# Common Index

common_index = df_returns.index.intersection(df_weights.index)  # Fechas en común
df_returns_reindex = df_returns.reindex(common_index)
df_weights = df_weights.reindex(common_index)

df_returns_reindex

In [11]:
# Create the Portfolio Returns

df_weighted_returns = df_returns_reindex * df_weights

df_weighted_returns

In [12]:
# Create the Portfolio Returns
df_returns_portfolio = df_returns.copy()

# Add the columns
df_returns_portfolio['Portfolio'] =  df_weighted_returns.sum(axis = 1)

df_returns_portfolio['Portfolio'].dropna()

In [13]:
# Time Series Graphs

df_plot = df_returns_portfolio.dropna()

plt.figure(figsize=(10, 6))
plt.plot(df_plot.cumsum(), label=df_plot.columns, alpha=1)

# Config
plt.title('Cumulative Returns Time Series')
plt.xlabel('Time Index')
plt.ylabel('Cumulative Returns')
plt.legend()

# Show
plt.grid(True)
plt.show()

### Define different rebalancing frequencies ###

In [14]:
# Create weights for different rebalancing frequencies

df_weights_5d = rolling_weights(df_returns, 0.0025, rebalance_freq=5)
df_weights_21d = rolling_weights(df_returns, 0.0025, rebalance_freq=21)
df_weights_63d = rolling_weights(df_returns, 0.0025, rebalance_freq=63)
df_weights_126d = rolling_weights(df_returns, 0.0025, rebalance_freq=126)
df_weights_252d = rolling_weights(df_returns, 0.0025, rebalance_freq=252)

In [15]:
# Create the Returns

df_weighted_returns_5d = df_returns_reindex * df_weights_5d
df_weighted_returns_21d = df_returns_reindex * df_weights_21d
df_weighted_returns_63d = df_returns_reindex * df_weights_63d
df_weighted_returns_126d = df_returns_reindex * df_weights_126d
df_weighted_returns_252d = df_returns_reindex * df_weights_252d

In [16]:
# Add the columns

df_returns_reindex['5d_port'] = df_weighted_returns_5d.sum(axis = 1)
df_returns_reindex['21d_port'] = df_weighted_returns_21d.sum(axis = 1)
df_returns_reindex['63d_port'] = df_weighted_returns_63d.sum(axis = 1)
df_returns_reindex['126d_port'] = df_weighted_returns_126d.sum(axis = 1)
df_returns_reindex['252d_port'] = df_weighted_returns_252d.sum(axis = 1)

In [17]:
# Time Series Graphs

plt.figure(figsize=(10, 6))
plt.plot(df_returns_reindex.cumsum(), label=df_returns_reindex.columns, alpha=1)

# Config
plt.title('Cumulative Returns Time Series')
plt.xlabel('Time Index')
plt.ylabel('Cumulative Returns')
plt.legend()

# Show
plt.grid(True)
plt.show()

### Comparing Portfolios ###

In [18]:
# Now the table
analytics_table = calculate_analytics(df_returns_reindex)

analytics_table